In [1]:
import pandas as pd
import os
import xmltodict
import xml.etree.ElementTree as ET

In [2]:
def read_excel(excel_path):
    try:
        d = pd.read_excel(excel_path, sheet_name=None)
        return d
    except Exception as r:
        print(str(r)+'\n')
        print(excel_path)

In [3]:
def write_excel(data_list,path):
    try:
        df = pd.DataFrame(data_list)
        order=[]
        for key in data_list[0].keys():
            order.append(key)
        df=df[order]
        df.to_excel(path)
    except Exception as reason:
        print(reason)
        pass

In [4]:
def read_csv(excel_path):
    try:
        d = pd.read_csv(excel_path)
        return d
    except Exception as r:
        print(str(r)+'\n')
        print(excel_path)

In [5]:
def write_csv(data_list,path):
    try:
        df = pd.DataFrame(data_list)
        order=[]
        for key in data_list[0].keys():
            order.append(key)
        df=df[order]       
        df.to_csv(path,index=False,sep=',',encoding='utf-8')
    except Exception as reason:
        print(reason)
        pass

In [14]:
def get_all_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

# 使用示例


In [19]:
folder_path = 'D:\\数据\\data\\test'
all_file_paths = get_all_file_paths(folder_path)
print(len(all_file_paths))
for path in all_file_paths:
    with open(path,encoding='utf-8') as xml_file:
        for line in xml_file:
            print(line)

1
<?xml version="1.0" encoding="UTF-8"?>

<records>

<!-- Copyright (c) 2019 Clarivate Analytics Web of Science  -->

<records xmlns="http://clarivate.com/schema/wok5.30/public/FullRecord">

<REC r_id_disclaimer="ResearcherID data provided by Clarivate Analytics"><UID>WOS:001216401200001</UID><static_data><summary><EWUID><WUID coll_id="WOS"/><edition value="WOS.SCI"/></EWUID><pub_info sortdate="2024-05-01" pubyear="2024" has_abstract="Y" coverdate="MAY 1 2024" pubmonth="MAY 1" vol="633" early_access_date="MAR 2024" early_access_month="03" early_access_year="2024" pubtype="Journal"><page page_count="6"/></pub_info><titles count="6"><title type="source">JOURNAL OF CRYSTAL GROWTH</title><title type="source_abbrev">J CRYST GROWTH</title><title type="abbrev_iso">J. Cryst. Growth</title><title type="abbrev_11">J CRYST GR</title><title type="abbrev_29">J CRYST GROWTH</title><title type="item">Advancements in Iso-Diameter seeded growth of CdZnTe crystals: A numerical modeling approach and appl

In [30]:
keys=['UT (Unique WOS ID)','DOI','Authors','Author Full Names','Article Title','Author Keywords','Keywords Plus','Abstract','Addresses',
      'Affiliations','Reprint Addresses','Email Addresses','ORCIDs','Source Title','Journal Abbreviation','ISSN','Publication Year',
      'Document Type','WoS Categories','Web of Science Index']

In [31]:
global WoS
WoS={}

In [51]:
def jiexi_xml(xml_path,PUB_Y):
    global WoS
    try:    
        with open(xml_path,encoding='utf-8') as xml_file:
            for line in xml_file:
                xml_content = line  # 去掉行末的换行符
                if '<UID>' not in xml_content:
                    continue
                temp={}
                for key in keys:
                    temp[key]=''    
                index= xml_content.find('<UID>')
                lastindex=xml_content.find('</UID>')
                UT=xml_content[index+5:lastindex]
                temp['UT (Unique WOS ID)']=UT
#                 print(UT)
            
                index=xml_content.find('<EWUID>')
                lastindex=xml_content.find('</EWUID>')
                Edition=xml_content[index+7:lastindex].replace('<WUID coll_id="WOS"/>','')  
                Edition=Edition.replace('"/><edition value="WOS.',';')
                Edition=Edition.replace('<edition value="WOS.','')
                Edition=Edition.replace('"/>','')
                temp['Web of Science Index']=Edition
#                 print(Edition)
            
                index=xml_content.find('<pub_info')
                lastindex=xml_content.find('</pub_info')                
                pubdate=xml_content[index+9:lastindex]
                pubdate=pubdate[pubdate.index('pubyear="')+9:]
                pubyear=pubdate[:pubdate.index('" ')]
                temp['Publication Year']=pubyear
#                 print(pubyear)
                
                index=xml_content.find('<title type="source">')
                source=xml_content[index+len('<title type="source">'):]
                lastindex=source.find('</title>')
                temp['Source Title']=source[:lastindex]
#                 print(source[:lastindex])
                
                index=xml_content.find('<title type="source_abbrev">')
                source=xml_content[index+len('<title type="source_abbrev">'):]
                lastindex=source.find('</title>')
                temp['Journal Abbreviation']=source[:lastindex]
#                 print(source[:lastindex])
                
                index=xml_content.find('<title type="item">')
                source=xml_content[index+len('<title type="item">'):]
                lastindex=source.find('</title>')
                temp['Article Title']=source[:lastindex]
#                 print(source[:lastindex])
                
                index=xml_content.find('<names count=')
                source=xml_content[index+len('<names count='):]
                lastindex=source.find('</names>')
                authors=source[:lastindex].split('</name>')
#                 print(authors)
                for i in range(len(authors)-1):  
                    author=authors[i]
#                     print(author)
                    if 'role="author"' not in author:
                        continue
                    fullname=author[author.index('<full_name>')+11:author.index('</full_name>')]
                    temp['Author Full Names']=temp['Author Full Names']+fullname+';'
                    wosname=author[author.index('<wos_standard>')+14:author.index('</wos_standard>')]
                    temp['Authors']=temp['Authors']+ wosname+';'
                    if 'orcid_id_tr' in author:
                        index=author.index('orcid_id_tr="')
                        ORCID=author[index+13:]
                        index=ORCID.index('"')
                        ORCID='['+fullname+']'+ORCID[:index] 
                        temp['ORCIDs']=temp['ORCIDs']+ORCID+';'
                    else:
                        ORCID=''
                    if '<email_addr>' in author:
                        index=author.index('<email_addr>')
                        email=author[index+12:]
                        index=email.index('</email_addr>')
                        email='['+fullname+']'+email[:index]
                        temp['Email Addresses']=temp['Email Addresses']+email+';'
                    else:
                        email=''
#                     print('email')    
                        
                if '<doctypes' in xml_content:    
                    index=xml_content.find('<doctypes')
                    source=xml_content[index+len('<doctypes'):]
                    lastindex=source.find('</doctypes>')
                    dtype=source[source.index('<doctype>')+9:lastindex]
                    dtype=dtype.replace('</doctype>',';')
                    dtype=dtype.replace('<doctype>','')
                    temp['Document Type']=dtype
#                     print('<doctypes')
                
                if '<addresses count=' in xml_content:
                    index=xml_content.find('<addresses count=')
                    source=xml_content[index+len('<addresses count='):]
                    lastindex=source.find('</addresses>')
                    addresses=source[:lastindex].split('</address_name>')
                
                    for i in range(len(addresses)-1):
                        address=addresses[i]
                        full_address=address[address.index('<full_address>')+14:address.index('</full_address>')]
                    
                        if '<organization pref="Y">' in address:
                            index=address.index('<organization pref="Y">')+len('<organization pref="Y">')
                            affiliation=address[index:]
                            lastindex=affiliation.index('</organization>')
                            temp['Affiliations']=temp['Affiliations']+affiliation[:lastindex]+';'                        
                        elif '<organization>' in full_address:
                            index=address.index('<organization>')+len('<organization>')
                            affiliation=address[index:]
                            lastindex=affiliation.index('</organization>')
                            temp['Affiliations']=temp['Affiliations']+affiliation[:lastindex]+';'                    
                    
                        temp['Addresses']=temp['Addresses']+ '['
                        if '<names count' in address: 
                            authors=address[address.index('<names count'):address.index('</names>')].split('</name>')                     
                            for j in range(len(authors)-1):
                                author=authors[j]
                                temp['Addresses']=temp['Addresses']+author[author.index('<full_name>')+11:author.index('</full_name>')]+';'
                        temp['Addresses']=temp['Addresses']+']'+full_address+'; '
#                     print('<addresses count=')
                    
                if '<reprint_addresses' in xml_content:
                    index=xml_content.find('<reprint_addresses')
                    source=xml_content[index+len('<reprint_addresses'):]
                    lastindex=source.find('</reprint_addresses>')
                    reprints=source[:lastindex].split('</address_name>')
                
                    for i in range(len(reprints)-1):
                        reprint=reprints[i]
                        full_address=reprint[reprint.index('<full_address>')+14:reprint.index('</full_address>')]                     
                        if '<names count' in reprint: 
                            authors=reprint[reprint.index('<names count'):reprint.index('</names>')].split('</name>')                     
                            for j in range(len(authors)-1):
                                author=authors[j]
                                temp['Reprint Addresses']=temp['Reprint Addresses']+author[author.index('<full_name>')+11:author.index('</full_name>')]+';'
                    
                        temp['Reprint Addresses']=temp['Reprint Addresses'].strip(';')+'(corresponding author), '+full_address+'; '
#                     print('<reprint_addresses')
                
                if '<category_info>' in xml_content:
                    index=xml_content.find('<category_info>')
                    source=xml_content[index+len('<category_info>'):]
                    lastindex=source.find('</category_info>')
                    subjects=source[:lastindex]
                    subjects=subjects[subjects.index('<subjects'):subjects.index('</subjects>')].split('</subject>')
                    for subject in subjects:
                        if '<subject ascatype="traditional">' in subject:
                            temp['WoS Categories']=temp['WoS Categories']+subject[subject.index('<subject ascatype="traditional">')+len('<subject ascatype="traditional">'):]+';'
#                     print('<category_info>')
                    
                if '<keywords' in xml_content:
                    index=xml_content.find('<keywords')
                    source=xml_content[index+len('<keywords'):]
                    index=source.find('<keyword>')
                    source=source[index+len('<keyword>'):]
                    lastindex=source.find('</keywords>')
                    keywords=source[:lastindex].split('</keyword>')
                    for keyword in keywords:
                        temp['Author Keywords']=temp['Author Keywords']+keyword.replace('<keyword>','')+';'
                    temp['Author Keywords']=temp['Author Keywords'].strip(';')
#                     print('<keywords')
                
                if '<keywords_plus' in xml_content:
                    index=xml_content.find('<keywords_plus')
                    source=xml_content[index+len('<keywords_plus'):]
                    index=source.find('<keyword>')
                    source=source[index+len('<keyword>'):]
                    lastindex=source.find('</keywords_plus>')
                    keywords=source[:lastindex].split('</keyword>')
                    for keyword in keywords:
                        temp['Keywords Plus']=temp['Keywords Plus']+keyword.replace('<keyword>','')+';'
                    temp['Keywords Plus']=temp['Keywords Plus'].strip(';')
#                     print( '<keywords_plus')
               
                index=xml_content.find('<identifiers>')
                source=xml_content[index+len('<identifiers>'):]
                index=source.find('<identifier value="')
                source=source[index+len('<identifier value="'):]                
                lastindex=source.find('</identifiers>')
                identifiers=source[:lastindex].split('/>')
                for identify in identifiers:
                    if 'type="issn"' in identify:
                        temp['ISSN']=identify.replace('<identifier type="issn" value="','').replace('"','')
                    if 'type="doi"' in identify:
                        temp['DOI']=identify.replace('<identifier type="doi" value="','').replace('"','')
                        
                if '<abstract_text count' in xml_content:
                    index=xml_content.find('<abstract_text count')
                    source=xml_content[index+len('<abstract_text count'):]
                    index=source.find('>')
                    source=source[index+1:]                
                    lastindex=source.find('</abstract_text>')
                    temp['Abstract']=source[:lastindex] 
#                     print('<abstract_text count')
            
#                 print(temp)
                if UT not in WoS.keys():
                    if 'china' in temp['Addresses'].lower() and int(temp['Publication Year'])==int(PUB_Y): 
#                     if int(temp['Publication Year'])==int(PUB_Y):     
                        WoS[UT]=temp
#                 else:
#                     print(UT+' AAAAAA')
            
    except Exception as e:  # 捕获任何异常
#         print(e)
        pass
#         print(UT)
#         print(full_address)
#         print(f"发生错误: {e}") 
        
     

In [55]:
PUB_Y=2024
folder_path = 'F:\\2024'
all_file_paths = get_all_file_paths(folder_path)
print(len(all_file_paths))
global WoS
wancheng=[]
WoS={}
for i in range(len(all_file_paths)):
    if i%1000==0:
        print(i)
    xml_path=all_file_paths[i]
    if 'xml' in xml_path:
#         print(xml_path)
        jiexi_xml(xml_path,PUB_Y)
#         temp1={}
#         temp1['wancheng']=xml_path
#         wancheng.append(temp1)
    if i%10000==0:        
        rpath='D:\\数据\\中科院发文数据\\China\\CAS_data_'+str(PUB_Y)+'-'+str(int(i/10000))+'.xlsx'
        write_excel(list(WoS.values()),rpath)
        
        WoS={}
        
#         rpath='D:\\数据\\中科院发文数据\\China\\yiwancheng.xlsx'
#         write_excel(wancheng,rpath)

# xml_path='D:\\数据\\data\\fffeff94ee75882d5b8e09a12aaed95e.xml'
# PUB_Y=2024
# jiexi_xml(xml_path,PUB_Y)
# rpath='D:\\数据\\中科院发文数据\\China\\CAS_data_'+str(PUB_Y)+'.xlsx'
# write_excel(list(WoS.values()),rpath)


34932
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000


In [56]:
rpath='D:\\数据\\中科院发文数据\\China\\CAS_data_'+str(PUB_Y)+'.xlsx'
write_excel(list(WoS.values()),rpath)

In [ ]:
keys=['UT (Unique WOS ID)','DOI','Authors','Author Full Names','Article Title','Author Keywords','Keywords Plus','Abstract','Addresses',
      'Affiliations','Reprint Addresses','Email Addresses','ORCIDs','Source Title','Journal Abbreviation','ISSN','Publication Year',
      'Document Type','WoS Categories','Times_Cited','Web of Science Index']

for year in range(2024,2025):
    print(year)
    citation={}
    rpath='D:\\数据\\data\\Times_Cited_'+str(year)+'.csv'
    data=read_csv(rpath)
    for i in range(0,len(data['Accession_Number'])):
        UT=data['Accession_Number'][i]
        citation[UT]=data['Times_Cited'][i]
    for s in range(1,5):
        
        CAS={}    
        rpath='D:\\数据\\中科院发文数据\\China\\CAS_data_'+str(year)+'-'+str(s)+'.xlsx'
        df=read_excel(rpath)
        for sheet in df:
            data=df[sheet]
            for i in range(0,len(data['UT (Unique WOS ID)'])):
                UT=data['UT (Unique WOS ID)'][i]
                CAS[UT]={}
                for key in keys:
                    CAS[UT][key]=''
                for key in data.keys():
                    if key in keys:
                        CAS[UT][key]=data[key][i]
                if UT in citation.keys():
                    CAS[UT]['Times_Cited']=citation[UT] 
                else:
                    CAS[UT]['Times_Cited']=0
        rpath='D:\\数据\\data\\CAS_data_'+str(year)+'-'+str(s)+'.xlsx'
        write_excel(list(CAS.values()),rpath)
    
    

2024
